<a href="https://colab.research.google.com/github/lrdplopes/llm-notebooks/blob/main/notebooks/rag_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q python-dotenv

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

False

In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# This is the YouTube video we're going to use.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"

## Setting up the model
Define the LLM that will be part of the workflow

In [4]:
!pip install -q langchain openai langchain_openai

In [5]:
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

### Testing the model

In [6]:
model.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

AIMessage(content='The Los Angeles Dodgers won the 2020 World Series during the COVID-19 pandemic.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 21, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fda93848-2300-408b-b259-5f536ad8458b-0')

### Transforming it in `STR`

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = model | parser
chain.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

'The Los Angeles Dodgers won the World Series during the COVID-19 pandemic in 2020. They defeated the Tampa Bay Rays in a six-game series to claim their first championship since 1988.'

## Introducing `prompt` templates

In [8]:
from langchain.prompts import ChatPromptTemplate
template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Lucas' is Gabriella's husband", question="Who is Lucas?")

'Human: \nAnswer the question based on the context below. If you can\'t\nanswer the question, reply "I don\'t know".\n\nContext: Lucas\' is Gabriella\'s husband\nQuestion: Who is Lucas?\n'

In [9]:
chain = prompt | model | parser
chain.invoke({
    "question": "Who is Lucas?",
    "context": "Lucas' is Gabriella's husband"
})

"Lucas is Gabriella's husband."

## Combining `Chains`
We can combine different chains to create more complex workflows. For example, let's create a second chain that translates the answer from the first chain into a different language.

Let's start by creating a new prompt template for the translation chain.

In [10]:
# Create a `translation` chain
translation_prompt = ChatPromptTemplate.from_template(
    "Translate the {answer} to the {language}"
)

In [11]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Lucas' is Gabriella's husband",
        "question": "Who is Lucas?",
        "language": "French",
    }
)

'Lucas est le mari de Gabriella.'

## Transcribing the YouTube Video
The context we want to send the model comes from a YouTube video. Let's download the video and transcribe it using OpenAI's Whisper - https://openai.com/research/whisper

In [12]:
!pip install -q whisper pytube openai-whisper

In [13]:
import tempfile
import whisper
from pytube import YouTube

# create an `if` only if we haven't created the transcription file yet.
if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()

    # load the `base` model
    whisper_model = whisper.load_model("base")
    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()
        with open("transcription.txt", "w") as file:
            file.write(transcription)

- Let's read the transcription and display the first few characters to ensure everything works as expected.

In [14]:
with open("transcription.txt", "r") as file:
    transcription = file.read()

transcription[:100]

"I think it's possible that physics has exploits and we should be trying to find them. arranging some"

## Using the entire transcription as context
If we try to invoke the chain using the transcription as context, the model will return an error because the context is too long.

Large Language Models support limitted context sizes. The video we are using is too long for the model to handle, so we need to find a different solution.

In [15]:
try:
  chain.invoke({
      "context": transcription,
      "question": "Is reading papers a good idea?"
  })
except Exception as e:
  print(e)

Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 47047 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


## Splitting the transcription
Since we can't use the entire transcription as the context for the model, a potential solution is to split the transcription into smaller chunks. We can then invoke the model using only the relevant chunks to answer a particular question.

- Let's start loading the transcription in `memory`

In [16]:
!pip install -q langchain_community

In [17]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt")
text_documents = loader.load()

There are many different ways to split a document. For this example, we'll use a simple splitter that splits the document into chunks of a fixed size - https://python.langchain.com/docs/modules/data_connection/document_transformers/.

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
documents = text_splitter.split_documents(text_documents)

## Finding the relevant chunks
Given a particular question, we must find the relevant chunks from the transcription to send to the model. Here is where the idea of `embeddings` comes in - https://dashboard.cohere.com/playground/embed.

In [19]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
embedded_query = embeddings.embed_query("Who is Gabriella")

print(f"Embedding lenght: {len(embedded_query)}")
print(embedded_query[:10])

Embedding lenght: 1536
[-0.013011531379663712, 0.0048802254384083185, -0.022562932928235913, -0.01819451768934817, -0.001633649975826097, -0.0026779964937517775, -0.0131773291548687, -0.005561438562932889, -0.008289895279507574, -0.0315736877355085]


In [20]:
sentence_1 = embeddings.embed_query("Lucas' hobbie is read")
sentence_2 = embeddings.embed_query("Gabriella' hobbie is stay in the phone")


We can now compute the similarity between the query and each of the two sentences. The closer the embeddings are, the more similar the sentences will be.

We can use Cosine Similarity to calculate the similarity between the query and each of the sentences - https://en.wikipedia.org/wiki/Cosine_similarity.

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence_1 = cosine_similarity([embedded_query], [sentence_1])[0][0]
query_sentence_2 = cosine_similarity([embedded_query], [sentence_2])[0][0]

query_sentence_1, query_sentence_2

(0.7365613597765673, 0.8350747678086314)

## Setting up a Vector Store
We need an efficient way to store document chunks, their embeddings, and perform similarity searches at scale. To do this, we'll use a vector store. A vector store is a database of embeddings that specializes in fast similarity searches.

To understand how it works, first let's create the `vector-store` in memory.

In [25]:
!pip install --upgrade --quiet  "docarray"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 5.8 MB/s eta 0:00:00


In [26]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Mary's sister is Susana",
        "John and Tommy are brothers",
        "Patricia likes white cars",
        "Pedro's mother is a teacher",
        "Lucia drives an Audi",
        "Mary has two siblings",
    ],
    embedding=embeddings,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [27]:
vectorstore1.similarity_search_with_score("Who likes Audi", k=3)

[(Document(page_content='Lucia drives an Audi'), 0.858028343515171),
 (Document(page_content='Patricia likes white cars'), 0.8273385734903588),
 (Document(page_content='Mary has two siblings'), 0.7164998143349375)]

## Connecting the vector store to the chain
We can use the vector store to find the most relevant chunks from the transcription to send to the model.

We need to configure a Retriever. The retriever will run a similarity search in the vector store and return the most similar documents back to the next step in the chain - https://python.langchain.com/docs/modules/data_connection/retrievers/.

In [28]:
retriever_1 = vectorstore1.as_retriever()
retriever_1.invoke("Who likes Audi")

[Document(page_content='Lucia drives an Audi'),
 Document(page_content='Patricia likes white cars'),
 Document(page_content='Mary has two siblings'),
 Document(page_content="Pedro's mother is a teacher")]

Our prompt expects two parameters, `"context"` and `"question"`. We can use the retriever to find the chunks we'll use as the context to answer the question.

We can create a map with the two inputs by using the `RunnableParallel` and `RunnablePassthrough` classes. This will allow us to pass the context and question to the prompt as a map with the keys "context" and "question."

In [29]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
setup = RunnableParallel(context=retriever_1, question=RunnablePassthrough())
setup.invoke("What color is Lucas car?")

{'context': [Document(page_content='Lucia drives an Audi'),
  Document(page_content='Patricia likes white cars'),
  Document(page_content='John and Tommy are brothers'),
  Document(page_content="Mary's sister is Susana")],
 'question': 'What color is Lucas car?'}

In [30]:
chain = setup | prompt | model | parser
chain.invoke("What color is Lucas car?")

"I don't know."

## Loading transcription into the vector store
We initialized the vector store with a few random strings. Let's create a new vector store using the chunks from the video transcription.

In [31]:
vectorstore_2 = DocArrayInMemorySearch.from_documents(
    documents,
    embeddings
)

In [33]:
chain = (
    {"context": vectorstore_2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What is the essential message from the video?")

'The essential message from the video is about the importance of collecting ground truth data for training neural networks, different mechanisms for collecting training data such as human annotation and simulation, and the significance of incremental product improvement and generating revenue along the way.'